<a href="https://colab.research.google.com/github/divakaivan/full-story-behind-multicollinearity/blob/main/The_full_story_behind_multicollinearity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear regression model

$\hat{y} = Xw + b$

The **goal** is to find the weights $w$ and bias $b$ such that, given features of a new data sample from the same distribution as $X$, the new sample's label will be predicted correctly with the smallest error.

# Loss

Our goal is: $w^*, b^* = {argmin}_{w, b} L(w, b)$

After we differentialte $||y - Xw||^2$ w.r.t. $w$, we end up with

# Optimal parameters

$w^* = (X^TX)^{-1} X^Ty$

Solving for $w^*$ provides us with the optimal solution for the optimization problem. However, it only works when the matrix $X^TX$ is invertible (has full rank - the columns are linearly independent).

**IF** it does not have full rank -> some of the features are linearly dependent. In a perfect world $X^TX$ would be a diagonal (or nearly diagonal) square matrix (i.e. no linearly dependent variables).

# Why do we need it to have full rank?

In $w^*$ we have $(X^TX)^{-1}$ - inversing the matrix. To inverse a matrix, the matrix needs to satisfy:

1. Be a square matrix - $(X^TX)^{-1}$ returns a square matrix so we are *good*
2. Have non-zero determinant. In the case of square matrices having a non-zero determinant is equivalent to being full rank
  * if features are linearly independent -> we are *good*
  * if features are linearly dependent -> we are *not* good

# What do we do when there is linear dependence or multicullinearity ?

We cannot directly solve for $w^*$. We need to employ alternative methods to estimate it. We need to get an invertible $X^TX$, so we can:
* use QR decomposition - useful even when $X^TX$ is not invertible
* SVD = obtain a pseudo-inverse $X^TX$
* regularization - helps mitigate multicollinearity
* other methods

Even though there is multicollinearity, we can still run a regression and obtain w estimates. Any if you ask an LLM, read a textbook, most likely it will leave at: thes estimates are bad and unreliable.

# Why ?

Multicollinearity means 'near-singularity' of the design matrix (matrix showing relationships between Xs and y).

# Why do we care for near-singularity (which is equivalent to near 0 determinant) ?

It means that we might have a near 0 eigenvalue. For a square matrix to have a determinant of (near) 0, at least one of its rows or columns must be a linear combination of the other rows or columns. This loss of dimensionality leads to a collapsing or compression of space, which is reflected in the eigenvalue being (near) 0.

# Why do we care if there is a (near) 0 eigenvalue ?

It suggests there is some strong linear relationship or multicollinearity among the features because of the way they relate to the `rank` and `condition number` of the matrix $X^TX$.

# Example of a matrix's eigenvalues when **there is** multicollinearity

In [1]:
import numpy as np
np.set_printoptions(suppress=True)
np.random.seed(42)

In [2]:
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([2, 4, 6, 8, 10])

X = np.column_stack((x1, x2))

XTX = np.dot(X.T, X)

eigenvalues_XTX = np.linalg.eigvals(XTX)

print('Multicollinearity case:')
print("X^TX:")
print(XTX)
print("Eigenvalues of X^TX:")
print(eigenvalues_XTX)

Multicollinearity case:
X^TX:
[[ 55 110]
 [110 220]]
Eigenvalues of X^TX:
[  0. 275.]


# Example of a matrix's eigenvalues when there is **no** multicollinearity

In [3]:
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([3, 5, 4, 6, 7])

X = np.column_stack((x1, x2))

XTX = np.dot(X.T, X)

eigenvalues_XTX = np.linalg.eigvals(XTX)

print('No multicollinearity case:')
print("X^TX:")
print(XTX)
print("Eigenvalues of X^TX:")
print(eigenvalues_XTX)

No multicollinearity case:
X^TX:
[[ 55  84]
 [ 84 135]]
Eigenvalues of X^TX:
[  1.9623732 188.0376268]


# How do eigenvalues relate to rank ?

On the one hand, if a matrix has full rank (from the start: this is one of the conditions in order to inverse a matrix) that means all its eigenvalues are non-zero. Each eigenvalue of a matrix corresponds to a factor by which a non-zero eigenvector is stretched or shrunk when the matrix is applied to it. If all rows and columns are linearly independent, then the matrix will not collapse any direction onto another during a transformation. Since there are no linearly dependent directions to collapse onto zero, all the eigenvalues must be non-zero.

On the other hand, a matrix with less than full rank means there is some linear dependence - there exists one direction in the input space that is mapped to 0 in the output space -> at least one 0 eigenvalue.

# Example of a matrix's rank when **there is** multicollinearity

In [4]:
# same matrix as before
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([2, 4, 6, 8, 10])

X = np.column_stack((x1, x2))

XTX = np.dot(X.T, X)

eigenvalues = np.linalg.eigvals(XTX)

print('Multicollinearity case:')
print(f'{eigenvalues=}')

rank = np.linalg.matrix_rank(XTX)
print("rank of the matrix:", rank) # supposed to be 3

Multicollinearity case:
eigenvalues=array([  0., 275.])
rank of the matrix: 1


# Example of a matrix's eigenvalues when there is **no** multicollinearity

In [5]:
# same matrix as before
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([3, 5, 4, 6, 7])

X = np.column_stack((x1, x2))

XTX = np.dot(X.T, X)

eigenvalues_XTX = np.linalg.eigvals(XTX)

print('No multicollinearity case:')
print("X^TX:")
print(XTX)
print("Eigenvalues of X^TX:")
print(eigenvalues_XTX)

rank = np.linalg.matrix_rank(XTX)
print("rank of the matrix:", rank) # supposed to be 2

No multicollinearity case:
X^TX:
[[ 55  84]
 [ 84 135]]
Eigenvalues of X^TX:
[  1.9623732 188.0376268]
rank of the matrix: 2


# How do eigenvalues relate to the matrix's condition number ?

The condition number of a matrix is a measure of its sensitivity to pertributions or changes in the elements of the matrix; or how small changes in the elements of the matrix might affect its behaviour or properties, such as its eigenvalues or the solutions to linear systems involving that matrix.

The spread of the eigenvalues can give insight into the matrix's conditioning - smaller spread generally suggests better conditioning.

In most cases, a larger condition number indicates poorer conditioning, while a smaller - better conditioning.

# Example of a matrix's rank when **there is** multicollinearity

In [6]:
# same matrix as before
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([2, 4, 6, 8, 10])

X = np.column_stack((x1, x2))

XTX = np.dot(X.T, X)

eigenvalues = np.linalg.eigvals(XTX)

print('Multicollinearity case:')
print(f'{eigenvalues=}')

rank = np.linalg.matrix_rank(XTX)
print("rank of the matrix:", rank) # supposed to be 2

condition_number = np.linalg.cond(XTX)
print(f'{condition_number=}')

Multicollinearity case:
eigenvalues=array([  0., 275.])
rank of the matrix: 1
condition_number=inf


# Example of a matrix's eigenvalues when there is **no** multicollinearity

In [7]:
# same matrix as before
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([3, 5, 4, 6, 7])

X = np.column_stack((x1, x2))

XTX = np.dot(X.T, X)

eigenvalues_XTX = np.linalg.eigvals(XTX)

print('No multicollinearity case:')
print("X^TX:")
print(XTX)
print("Eigenvalues of X^TX:")
print(eigenvalues_XTX)

rank = np.linalg.matrix_rank(XTX)
print("rank of the matrix:", rank) # supposed to be 2

condition_number = np.linalg.cond(XTX)
print(f'{condition_number=}')

No multicollinearity case:
X^TX:
[[ 55  84]
 [ 84 135]]
Eigenvalues of X^TX:
[  1.9623732 188.0376268]
rank of the matrix: 2
condition_number=95.82154225314864


# Building up to multicollinearity

* Eigenvalues and Eigenvectors: when an eigenvalue is close to 0, it indicates that the associated eigenvector does not scale much during the applied transformation
* Minimal scaling: the vector does not scale (much), this means the transformation has very little effect along the direction of that eigenvector (little effect on the features or characteristics of the data that are aligned with that particular eigenvector)
* Redundancy: if an eigenvector is nearly parallel to the null space of the transformation matrix, this suggests there is a linear combination of the data (features in the design matrix) that has no effect on the outcome
* Multicollinearity: redundancy (or near-redundancy) in the data is a characteristic of multicollinearity - one predictor variable can be approximately expressed as a linear combination of the others. If we have multicollinearity, there is near linear dependence and the matrix is (near) singular, which makes $X^TX$ ill-conditioned and numerically unstable (sensitive to pertrubations) which inflates the parameter variances, resulting in poor estimates

# From a 0 eigenvalue to multicollinearity

A 0 eigenvalue indicates minimal scaling of the corresponding eigenvector which suggests redundancy or near-redundancy in the data. This redundancy implies a high degree of correlation between predictor variables, leading to multicollinearity in the regression model.


# Example of linear regression when **there is** multicollinearity

In [8]:
# same as before
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([2, 4, 6, 8, 10])

X = np.column_stack((x1, x2))

XTX = np.dot(X.T, X)

eigenvalues = np.linalg.eigvals(XTX)

print('Multicollinearity case:')
print(f'{eigenvalues=}')

rank = np.linalg.matrix_rank(XTX)
print("Rank of the matrix:", rank)

condition_number = np.linalg.cond(XTX)
print(f'{condition_number=}')

from sklearn.linear_model import LinearRegression

true_coef = [2, 1]

y = np.dot(X, true_coef) + np.random.normal(0, 0.1, size=len(x1))

model = LinearRegression()
model.fit(X, y)

print("\nOriginal Coefficients and Intercept:")
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

X_perturbed = X + 0.001 * np.random.randn(*X.shape)

model_perturbed = LinearRegression()
model_perturbed.fit(X_perturbed, y)

print("\nPerturbed Coefficients and Intercept:")
print("Coefficients:", model_perturbed.coef_)
print("Intercept:", model_perturbed.intercept_)

Multicollinearity case:
eigenvalues=array([  0., 275.])
Rank of the matrix: 1
condition_number=inf

Original Coefficients and Intercept:
Coefficients: [0.80039912 1.60079824]
Intercept: 0.03991352436920792

Perturbed Coefficients and Intercept:
Coefficients: [-8.6065131   6.30414128]
Intercept: 0.03940363891543086


# Example of linear regression when there is **no** multicollinearity

In [9]:
# same as before
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([3, 5, 4, 6, 7])

X = np.column_stack((x1, x2))

XTX = np.dot(X.T, X)

eigenvalues = np.linalg.eigvals(XTX)

print('No multicollinearity case:')
print(f'{eigenvalues=}')

rank = np.linalg.matrix_rank(XTX)
print("Rank of the matrix:", rank)

condition_number = np.linalg.cond(XTX)
print(f'{condition_number=}')

from sklearn.linear_model import LinearRegression

true_coef = [2, 1]

y = np.dot(X, true_coef) + np.random.normal(0, 0.1, size=len(x1))

model = LinearRegression()
model.fit(X, y)

print("\nOriginal Coefficients and Intercept:")
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

X_perturbed = X + 0.001 * np.random.randn(*X.shape)

model_perturbed = LinearRegression()
model_perturbed.fit(X_perturbed, y)

print("\nPerturbed Coefficients and Intercept:")
print("Coefficients:", model_perturbed.coef_)
print("Intercept:", model_perturbed.intercept_)

No multicollinearity case:
eigenvalues=array([  1.9623732, 188.0376268])
Rank of the matrix: 2
condition_number=95.82154225314864

Original Coefficients and Intercept:
Coefficients: [2.05446569 0.92175784]
Intercept: 0.15618973818845738

Perturbed Coefficients and Intercept:
Coefficients: [2.05486836 0.92235525]
Intercept: 0.1525767124126478
